In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

c:\Users\jaesc2\GitHub\skforecast


In [2]:
# Create a df with 100 cols
n_rows = 10000
n_cols = 100

df = pd.DataFrame(
    np.random.randn(n_rows, n_cols), 
    columns=[f"col_{i}" for i in range(n_cols)],
    index=pd.date_range("2023-01-01", periods=n_rows, freq="h")
)

In [3]:
# Convert df to long format
df_long = df.melt(ignore_index=False, var_name="col", value_name="value")
df_long

,col,value
2023-01-01 00:00:00,col_0,0.923967
2023-01-01 01:00:00,col_0,0.101799
2023-01-01 02:00:00,col_0,-1.006504
2023-01-01 03:00:00,col_0,-1.235637
2023-01-01 04:00:00,col_0,0.468410
...,...,...
2024-02-21 11:00:00,col_99,0.875016
2024-02-21 12:00:00,col_99,-0.022232
2024-02-21 13:00:00,col_99,-0.687094
2024-02-21 14:00:00,col_99,0.482631


In [4]:
set(df_long.select_dtypes(include=float).columns)

{'value'}

In [5]:
{
    col for col in df_long.columns 
    if pd.api.types.is_float_dtype(df_long[col])
}

{'value'}

In [7]:
from skforecast.preprocessing import exog_long_to_dict

exog_series_1 = pd.DataFrame({
        'exog_1': np.random.normal(0, 1, 10),
        'exog_2': np.random.choice(["A", "B", "C"], 10),
        'exog_3': np.random.randint(0, 10, 10),
        'series': 'series_1',
    }, index = pd.date_range(start='1-1-2000', periods=10, freq='D'))
exog_series_2 = pd.DataFrame({
    'exog_1': np.random.normal(0, 1, 10),
    'exog_2': np.random.choice(["A", "B"], 10),
    'exog_3': np.random.randint(0, 10, 10),
    'series': 'series_2',
}, index = pd.date_range(start='1-1-2000', periods=10, freq='D'))
exog_long = (
    pd.concat([exog_series_1, exog_series_2], axis=0)
    .reset_index()
    .rename(columns={"index": "datetime"})
)
exog_long = exog_long.loc[
    [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], :
].copy()
exog_long["exog_2"] = exog_long["exog_2"].astype("category")
exog_long["series"] = exog_long["series"].astype("category")
exog_dict = exog_long_to_dict(
    data=exog_long,
    series_id="series",
    index="datetime",
    freq="D",
    consolidate_dtypes=True,
    suppress_warnings=True,
)

In [10]:
exog_long

,datetime,exog_1,exog_2,exog_3,series
0,2000-01-01,1.082434,C,2,series_1
1,2000-01-02,0.077143,B,2,series_1
2,2000-01-03,1.604305,B,1,series_1
3,2000-01-04,-1.811204,A,3,series_1
4,2000-01-05,0.124763,A,7,series_1
6,2000-01-07,0.342647,A,7,series_1
7,2000-01-08,-1.211279,A,0,series_1
8,2000-01-09,-0.744761,B,5,series_1
9,2000-01-10,1.056954,C,9,series_1
10,2000-01-01,-0.167325,A,1,series_2


In [9]:
exog_dict['series_1'].dtypes

exog_1     float64
exog_2    category
exog_3     float64
dtype: object